## Pydantic Basics

pydantic models are the foundation of data validation in python. They use python type annotations to define the structure and validate data at runtime. 

In [8]:
from pydantic import BaseModel
from dataclasses import dataclass


In [12]:
@dataclass
class Person1():
    name: str
    age: int    
    city: str   
person = Person1(name="Alice", age=30, city="New York")

In [13]:
Person1(name="Alice", age=30, city=12)

Person1(name='Alice', age=30, city=12)

In [9]:
class Person(BaseModel):
    name: str
    age: int    
    city: str

person = Person(name="Alice", age=30, city="New York")
print(person)  # Output: name='Alice' age=30 city='New York'

name='Alice' age=30 city='New York'


In [10]:
person = Person(name="Alice", age=30, city=12)

ValidationError: 1 validation error for Person
city
  Input should be a valid string [type=string_type, input_value=12, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type

## Model with Optional Fields
add optional fields using python optional type

- pydantic validates types even for optional fields when values are provided

In [14]:
from typing import Optional

class Employee(BaseModel):
    id:int
    name: str
    department: str
    salary: Optional[float] = None  # Optional field with default value None
    is_active: Optional[bool] = True  # Optional field with default value True

emp1 = Employee(id=1, name="Bob", department="HR")
print(emp1)  # Output: id=1 name='Bob' department='HR' salary=None is_active=True

id=1 name='Bob' department='HR' salary=None is_active=True


In [15]:
emp2 = Employee(id=2, name="Alice", department="IT", salary=75000.0, is_active=False)
print(emp2)  # Output: id=2 name='Alice' department='IT'

id=2 name='Alice' department='IT' salary=75000.0 is_active=False


In [16]:
class Classroom(BaseModel):
    room_number: str
    students: list[str]  # List of student names
    capacity: int

classroom = Classroom(room_number="101A", students=["Alice", "Bob", "Charlie"], capacity=30)
print(classroom)  # Output: room_number='101A' students=['Alice', '

room_number='101A' students=['Alice', 'Bob', 'Charlie'] capacity=30


In [17]:
try:
    invalid_classroom = Classroom(room_number="102B", students=["Dave", 123, "Eve"], capacity=25)
except ValueError as e:
    print(f"Validation Error: {e}")

Validation Error: 1 validation error for Classroom
students.1
  Input should be a valid string [type=string_type, input_value=123, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type


## Model with Nested Models
Create complex structures with nested models:

In [18]:
class Address(BaseModel):
    street: str
    city: str
    zip_code: str

class Customer(BaseModel):
    id: int
    name: str
    address: Address  # Nested model

customer = Customer(
    id=1,
    name="John Doe",
    address=Address(street="123 Main St", city="Anytown", zip_code="12345")
)

print(customer)

id=1 name='John Doe' address=Address(street='123 Main St', city='Anytown', zip_code='12345')


## Pydantic Field customization and Constraints

THe field function in pydantic enhances model fields beyond basic type hints by allowing you to specify validation rules, default values, aliases, and more. 

In [19]:
from pydantic import Field

class Item(BaseModel):
    name: str = Field(min_length=2, max_length=50)
    price: float = Field(gt=0, le=1000)
    quantity: Optional[int] = Field(ge=0)

item = Item(name="Laptop", price=999.99, quantity=10)
print(item)


name='Laptop' price=999.99 quantity=10


In [20]:
item = Item(name="L", price=1500, quantity=-5)  # This will raise a validation error
print(item)

ValidationError: 3 validation errors for Item
name
  String should have at least 2 characters [type=string_too_short, input_value='L', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/string_too_short
price
  Input should be less than or equal to 1000 [type=less_than_equal, input_value=1500, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/less_than_equal
quantity
  Input should be greater than or equal to 0 [type=greater_than_equal, input_value=-5, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/greater_than_equal

In [21]:
class User(BaseModel):
    username: str = Field(..., description="The user's username")
    age:int = Field(default=18, description="The user's age, default is 18")
    email:str=Field(default_factory=lambda:"user@example.com", description="The user's email address")


user = User(username="johndoe")
print(user)

user2 = User(username="janedoe", age=25, email="jane@example.com")
print(user2)


username='johndoe' age=18 email='user@example.com'
username='janedoe' age=25 email='jane@example.com'


In [22]:
print(User.schema_json(indent=2))

{
  "properties": {
    "username": {
      "description": "The user's username",
      "title": "Username",
      "type": "string"
    },
    "age": {
      "default": 18,
      "description": "The user's age, default is 18",
      "title": "Age",
      "type": "integer"
    },
    "email": {
      "description": "The user's email address",
      "title": "Email",
      "type": "string"
    }
  },
  "required": [
    "username"
  ],
  "title": "User",
  "type": "object"
}


/tmp/ipykernel_72710/1709186597.py:1: PydanticDeprecatedSince20: The `schema_json` method is deprecated; use `model_json_schema` and json.dumps instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  print(User.schema_json(indent=2))
